In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import math
import matplotlib.pyplot as plt
import numpy as np

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    __R = 100@u_kΩ

    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__R__ = R
        SubCircuit.__init__(self, name, *self.__nodes__)
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )
        
    def wiper(self, w) :
        if w == 0 :
            self.R1.resistance = self.__R * 0.0000001
            self.R2.resistance = self.__R * 0.9999999
        elif w == 1 :
            self.R1.resistance = self.__R * 0.9999999
            self.R2.resistance = self.__R * 0.0000001
        else :
            self.R1.resistance = self.__R * w
            self.R2.resistance = self.__R * (1.0-w)

#Construction



# sawtooth oszillator

The heart of the VCO os a sawtooth oszillator. This type of oscillator is built with an integrator followed by a schmitt trigger. When the ramp has reached a certain threshold the schmitt trigger will flip and discharge the capacitor. In this DCO design the schmitt trigger is replaced with a digital timer that send a pulse to the integrator.

In [ ]:
d = schemdraw.Drawing(unit=2, inches_per_unit=0.6, lw=1.8)

d += ( U1 := elm.Opamp() )
d += ( L1 := elm.Line().left().length(d.unit/4).at(U1.in1) )
d += elm.Dot()
d += ( R1 := elm.Resistor().label("R4\n270k"))
d += elm.Tag().label("cv")
d += elm.Line().left().at(U1.in2).length(d.unit/4)
d += elm.Ground()
     
#Feedback
d += ( L2 := elm.Line().up().at(L1.end) )
d += elm.Dot()
d += elm.Capacitor().right().length(d.unit*1.5).label("C1\n1n")
d += elm.Dot()
d += ( L3 := elm.Line().down().toy(U1.out) )
d += elm.Dot()

#feedback transistor
d += elm.Line().up().at(L2.end)
d += elm.Line().right().length(d.unit*0.4)
d += ( Q1 := elm.BjtPnp(circle=True).anchor("collector").theta(-90) )
d += elm.Line().right().tox(L3.end)
d += elm.Line().down()

d += elm.Line().up().length(d.unit/4).at(Q1.base)
d += elm.Line().left().length(d.unit/2)
d += elm.Resistor().label("R3\n22k")
d += elm.Line().tox(R1.end)

# cv input amplifier 
d += ( Uin := elm.Opamp().right().anchor('out') )
d += elm.Line().left().length(d.unit/4).at(Uin.in2)
d += elm.Ground()
d += ( Lin :=  elm.Line().left().length(d.unit/4).at(Uin.in1) )
d += elm.Dot()
d += elm.Resistor().label('R1\n100k\n')
d += elm.Tag().label("pulse")

#feedback input amplifier
d += elm.Line().up().at(Lin.end)
d += elm.Resistor().right().length(d.unit*1.5).label('R2\n100k')
d += elm.Line().down().toy(Uin.out)
d += elm.Dot()

#output amplifier
d += elm.Line().right().at(U1.out).length(d.unit/2)
d += ( R5 := elm.Resistor().label('R5\n10k'))
d += ( Lout := elm.Line().right().length(d.unit/4) )
d += elm.Dot()
d += elm.Line().right().length(d.unit/4)
d += ( U3 := elm.Opamp().anchor('in2'))
d += elm.Line().down().at(Lout.end)
d += elm.Line().left().length(d.unit/4)
d += elm.Resistor().left().label('R6\n120k').label('+15V', loc='left')
#d += elm.Vdd().label('+15V')

#feedback output amplifier
d += elm.Line().left().at(U3.in1).length(d.unit/4)
d += ( Lref := elm.Line().up())
d += elm.Dot()
d += elm.Resistor().right().label('R8\n100k').length(d.unit*1.5)
d += elm.Line().down().toy(U3.out)
d += elm.Dot()

d += elm.Resistor().left().at(Lref.end).label('R7\n33k')
d += elm.Ground()

d += elm.Line().right().at(U3.out).length(d.unit/2)
d += elm.Tag().label("OUT")


d.draw()

C1 is the capcitor of the integrator. The CV input charges the capacitor until the pulse will open the transistor and the capacitor will be reset. U3 is the amplifying stage and will and does the level shifting.

In [ ]:
circuit = Circuit('cmos buffer')
circuit.include(spice_library['TL072'])
circuit.include(spice_library['BC556B'])
circuit.include(spice_library['J2N5459'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'vCharge', circuit.gnd, 'DC 0.594000V')
#circuit.V('5', 'vPulse', circuit.gnd, 'DC 0 PULSE ( -10 10V 0 0 0 1.135ms 1.1363636363636ms )')
circuit.V('5', 'vPulse', circuit.gnd, 'DC 0 PULSE ( 10 -10V 0 0 0 0.6ms 1.1363636363636ms )')
circuit.V('6', 'vRef', circuit.gnd, 'DC -5')

#pulse amplifier
circuit.C(10, 'vPulse', 'C10', 270@u_pF)
circuit.R(11, 'C10', circuit.gnd, 4.7@u_kΩ)

circuit.X(1, 'TL072', 'vRef', 'C10', '+15V', '-15V', 'xPulse')

#integrator
circuit.R(4, 'vCharge', 'R4', 270@u_kΩ)
circuit.X(2, 'TL072', circuit.gnd, 'R4', '+15V', '-15V', 'OUTi')
circuit.C(1, 'R4', 'OUTi', 1@u_nF)

circuit.R(3, 'xPulse', 'R3', 22@u_kΩ)
circuit.R(12, 'OUTi', 'R12', 270@u_Ω)
#circuit.BJT(1, 'R4', 'R3', 'OUTi', model="BC556B")
circuit.JFET(1, 'R4', 'R3', 'OUTi', model="J2N5459")

#output buffer
circuit.R(5, 'OUTi', 'R5', 10@u_kΩ)
circuit.R(6, 'R5', '+15V', 120@u_kΩ)

circuit.X(3, 'TL072', 'R5', 'X3', '+15V', '-15V', 'OUT')
circuit.R(8, 'X3', 'OUT', 100@u_kΩ)
circuit.R(7, 'X3', circuit.gnd, 33@u_kΩ)

print(circuit)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.initial_condition(OUTi=0.0)
analysis  = simulator.transient(step_time=1@u_us, start_time=10@u_ms, end_time=12@u_ms)

fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time (ms)')
ax1_buffer.set_ylabel('IN [V]')
buffer_axis, = ax1_buffer.plot(u_ms(analysis['OUT'].abscissa), analysis['OUT'], color='Red')
buffer_axis, = ax1_buffer.plot(u_ms(analysis['OUTi'].abscissa), analysis['OUTi'], color='Blue')
ax1_buffer.legend(('Vout [V]', 'Vintegtator'), loc=(0.7,0.8))

plt.tight_layout()
plt.show()


The blue signal is the output of the integrator. The red one the amplified one. The timer can be changed with the slides. With the original design the CV input controls the frequency. This means that the capacitor is charged until it has a defined voltage. with the digital input the capacitor will only be charged partialy. The CV voltage has to be adjusted to have the same voltage for all notes.

In [ ]:
dac = 2.048 * 1 * (2048 / ( 2 ** 12) ) 
print( dac )

In [ ]:
dac = 1.024 / (2048 / ( 2 ** 12) ) 
print( dac )

In [ ]:
notes = [ 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B'
]

octaves = [
    '-5', '-5 ', '-5', '-5 ', '-5', '-5', '-5 ', '-5', '-5', '-5', '-5 ', '-5', 
    '-4', '-4 ', '-4', '-4 ', '-4', '-4', '-4 ', '-4', '-4', '-4', '-4 ', '-4', 
    '-3', '-3 ', '-3', '-3 ', '-3', '-3', '-3 ', '-3', '-3', '-3', '-3 ', '-3', 
    '-2', '-2 ', '-2', '-2 ', '-2', '-2', '-2 ', '-2', '-2', '-2', '-2 ', '-2', 
    '-1', '-1 ', '-1', '-1 ', '-1', '-1', '-1 ', '-1', '-1', '-1', '-1 ', '-1', 
    '0', '0 ', '0', '0 ', '0', '0', '0 ', '0', '0', '0', '0 ', '0', 
    '1', '1 ', '1', '1 ', '1', '1', '1 ', '1', '1', '1', '1 ', '1', 
    '2', '2 ', '2', '2 ', '2', '2', '2 ', '2', '2', '2', '2 ', '2', 
    '3', '3 ', '3', '3 ', '3', '3', '3', '3', '3', '3', '3 ', '3', 
    '4', '4 ', '4', '4 ', '4', '4', '4 ', '4', '4', '4', '4 ', '4', 
    '5', '5 ', '5', '5 ', '5', '5', '5 ', '5', '5', '5', '5 ', '5'
]

In [ ]:
_dac_v_low = 2.048
_dac_v_high =  4.096

def calculate_dac(voltage) :
    result = []
    if voltage < _dac_v_low :
        result.append(0)
        result.append(0)
        result.append(0) 
        dac1_value = voltage / (_dac_v_low / 4095)
        result.append(dac1_value)
        result.append(0)
        result.append((_dac_v_low / 4095 * dac1_value))

    elif voltage <= _dac_v_high :
        result.append(0)
        result.append(_dac_v_low)
        result.append(0) 
        dac1_value = (voltage - _dac_v_low) / (_dac_v_low / 4095)
        result.append(4095)
        result.append(_dac_v_low)
        result.append((_dac_v_low / 4095 * dac1_value))

    elif voltage <= _dac_v_high + _dac_v_low :
        result.append(1)
        result.append(_dac_v_high)
        result.append(0) 
        dac1_value = (voltage - _dac_v_high) / (_dac_v_low / 4095)
        result.append(4095)
        result.append(_dac_v_high)
        result.append((_dac_v_low / 4095 * dac1_value))

    else :
        result.append(1)
        result.append(_dac_v_high)
        result.append(1) 
        dac1_value = (voltage - _dac_v_high) / (_dac_v_high / 4095)
        result.append(4095)
        result.append(_dac_v_high)
        result.append((_dac_v_high / 4095 * dac1_value))
        
    return result


In [ ]:
a = 440
v_step = 1 / 12

table = '''
<div class="section">
<div class="container">
<div class="content">
<table class="table">
  <thead>
    <tr>
      <th>Midi Note</th>
      <th>Octave></th>
      <th>Note</th>
      <th>Frequency [Hz]</th>
      <th>CV [V]</th>
      <th>time [ms]</th>
      <th>Scaling</th>
      <th>Final Frequency [Hz]</th>
      <th>Deviation [%]</th>
      <th>DAC [V]</th>
      <th> DAC1 Gain </th>
      <th> DAC2 Gain </th>
      <th> DAC1 value</th>
      <th> DAC2 value </th>
      <th> DAC1 [V] </th>
      <th> DAC2 [V] </th>
      <th> DAC1 real [V] </th>
      <th> DAC2 real </th>
    </tr>
  </thead>
  <tbody>
'''

cpu_freq = 48000000
prescaler = 48 * 2
timer_freq = cpu_freq / prescaler
R = 270@u_kΩ
C = 1@u_nF
#V = 4.422@u_V
V = 2.5@u_V

for s in np.arange(0, 128) :
    
    _freq = a * 2 ** ((s-69) / 12 )
    _time = 1 / _freq
    _scaling = timer_freq / _freq
    _round_scaling = int(_scaling)
    _real_freq = timer_freq/_round_scaling
    _deviation = _freq / _real_freq * 100
    #voltage = (float(val)/4096) * 3.3
    voltage = R * C * V / _time
        
    dac = voltage / (5/4095) 

    table += '<tr>'
    table += '<td>  %s  </td>' % s
    table += '<td>  %s </td>' % octaves[s]
    table += '<td> %s </td>' % notes[s]
    table += '<td> %.2f </td>' % _freq
    table += '<td> %.2f </td>' % ((s-69) * v_step)
    table += '<td> %.4f </td>' % (_time*1000)
    table += '<td> %d (%.2f) </td>' % ( _round_scaling, _scaling )
    table += '<td> %.2f </td>' % _real_freq
    table += '<td> %.4f </td>' % (100-_deviation)
    table += '<td> %f </td>' % voltage
    
    result = calculate_dac(voltage)
    table += '<td>%d </td>' % result[0]
    table += '<td>%d </td>' % result[2]
    table += '<td> %d </td>'  % result[1]
    table += '<td> %d </td>' % result[3]
    table += '<td> %.4f </td>' % result[4]
    table += '<td> %.4f </td>' % result[5]
    table += '<td> %.4f </td>' % (result[4] + result[5])
    table += '<td> %.2f </td>' % ( (result[4] + result[5]) / voltage * 100)
    table += '</tr>'

table += "</tbody></table></div></div></div>"

from IPython.display import display, HTML
display(HTML(table))

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<div class="hero is-medium" style="background: url('{{ '/assets/cmos_buffer_files/tmb_ube_screamer.jpg' | relative_url }}') no-repeat center center; background-size: cover; background-attachment: fixed;">
   <div class="hero-body">
        <div class="content has-text-centered">
        </div>
    </div>
</div>
'''))

In [ ]:
for i in np.arange( 0, 9, 1) :
    print( -33000 * ( (i/100000 ) + (-10/100000) ) )

In [ ]:
print( 3.3 / 4096)
print( 0.0833 / 3)

In [ ]:
old_val = 0
for i in np.arange( 0, 0.9167, 0.0833 ) :
    val = -33000 * ( (i/100000 ) + (-10/100000) );
    print( "CV: %f -> %f " % (val, val-old_val) )
    old_val = val
                   

In [ ]:
circuit = Circuit('cv buffer')
circuit.include(spice_library['TL072'])

circuit.V('1', '+3.3V', circuit.gnd, 'DC 3.3')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'vCharge', circuit.gnd, 'DC 0.594000V')
circuit.V('5', 'vPulse', circuit.gnd, 'DC 0 PULSE ( 0 5V 0 0 0 0.001ms 1.1363636363636ms )')

#pulse amplifier
circuit.R(1, 'vPulse', 'R1', 100@u_kΩ)
circuit.R(2, 'R1', 'xPulse', 100@u_kΩ)
circuit.X(1, 'TL072', circuit.gnd, 'R1', '+15V', '-15V', 'xPulse')


In [ ]:
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

circuit = parser.build_circuit(ground=5)
circuit.include(spice_library['TL072c'])
circuit.include(spice_library['J2N5459'])
circuit.include(spice_library['BC556B'])
circuit.include(spice_library['D1N4148'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', '/DAC_A', circuit.gnd, 'DC 0.594000V')
circuit.V('4', '/DAC_B', circuit.gnd, 'DC 0V')
circuit.V('5', '/pulse', circuit.gnd, 'DC 0 PULSE ( 10V -10V 0 0 0 0.6ms 1.1363636363636ms )')

for c in ( VoltageDivider(R=100@u_kΩ, w=0.318, name='saw_offset'),\
           VoltageDivider(R=100@u_kΩ, w=0.59, name='tri_offset'),\
           VoltageDivider(R=100@u_kΩ, w=0.4, name='sine_round'),\
           VoltageDivider(R=100@u_kΩ, w=0.5, name='sine_sim') ) :
    circuit.subcircuit(c)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.initial_condition(outi=0.0)
analysis  = simulator.transient(step_time=1@u_us, start_time=10@u_ms, end_time=12@u_ms)

fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
ax1_buffer.plot(analysis['SAW'].abscissa, analysis['SAW'], color='Red')
ax1_buffer.plot(analysis['TRI'].abscissa, analysis['TRI'], color='Blue')
ax1_buffer.plot(analysis['SINE'].abscissa, analysis['SINE'], color='Green')
ax1_buffer.legend(('Vin_a [V]', 'Vin_b', 'Vout [V]'), loc=(0.75,0.8))

plt.tight_layout()
plt.show()

print(f"SAW min: {min(analysis['SAW'])}, max: {max(analysis['SAW'])}")
print(f"TRI min: {min(analysis['TRI'])}, max: {max(analysis['TRI'])}")
print(f"SINE min: {min(analysis['SINE'])}, max: {max(analysis['SINE'])}")
print(f"OUTi min: {min(analysis['MES'])}, max: {max(analysis['MES'])}")



# calibration

# usage
